## Part 5 - Labelisation

Part 1

In [1]:
# Chargement des librairies nécessaires
import pandas as pd
import spacy

# Chargement de la dataset
dataset = pd.read_csv("dataset_CV.csv", delimiter = ";")

# Instanciation de la bibliothèque fr_core_news_lg de Spacy
nlp = spacy.load("fr_core_news_lg")
 
# Tokenization des textes de CV
dataset["Tokenization_preparation"] = dataset["CV_text_brut"].apply(lambda x : str(x).replace("\n", " "))
dataset["Tokenization"] = dataset["Tokenization_preparation"].apply(lambda x : nlp(x))
dataset["Number_of_tokens"] = dataset["Tokenization"].apply(lambda x : len(x))

In [2]:
dataset.head()

,CV_number,CV_text_brut,Tokenization_preparation,Tokenization,Number_of_tokens
0,CV_1,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE PRO...,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE PRO...,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIAS...",214
1,CV_2,Ananya Singh \nProfesseure de Lycée\nPROFIL PE...,Ananya Singh Professeure de Lycée PROFIL PERS...,"(Ananya, Singh, , Professeure, de, Lycée, PRO...",291
2,CV_3,"Samira Hadid \n16 place Urbain II, 63000 Clerm...","Samira Hadid 16 place Urbain II, 63000 Clermo...","(Samira, Hadid, , 16, place, Urbain, II, ,, 6...",315
3,CV_4,Carine Maurice \nIngénieure logiciel avec 10 a...,Carine Maurice Ingénieure logiciel avec 10 an...,"(Carine, Maurice, , Ingénieure, logiciel, ave...",292
4,CV_5,MARCELLE ANDRÉ EXPÉRIENCE PROFESSIONNELLE \nAS...,MARCELLE ANDRÉ EXPÉRIENCE PROFESSIONNELLE ASS...,"(MARCELLE, ANDRÉ, EXPÉRIENCE, PROFESSIONNELLE,...",284


In [3]:
dataset.shape

(31, 5)

In [5]:
## Création d'une dataframe pour accueillir les features des CV
df = pd.DataFrame(columns = ['CV_Sentences'])

## Extraction du texte du CV pour appliquer la sentencisation des CV
index = dataset.columns.get_loc("Tokenization_preparation")
doc = dataset.iloc[0,index]   ## CHANGER PAR UN i POUR BOUCLER SUR TOUS LES CV 
doc = nlp(doc)

In [6]:
## Création d'une liste de phrases qui va accueillir le texte découpé en phrases grâce à la fonction doc.sents
list_sentence = []
for sent in doc.sents :
    list_sentence.append(sent)

In [7]:
## Affectation des phrases dans la colonne CV_Sentences et suppression des parenthèses
df['CV_Sentences'] = list_sentence
df['Sentences_CV_clean'] = df['CV_Sentences'].apply(lambda x : str(x).replace("(","").replace(")",""))

In [8]:
df.head()

,CV_Sentences,Sentences_CV_clean
0,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIASTE)",SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE
1,(PROFIL),PROFIL
2,"(PERSONNEL, Je, suis, étudiante, au, lycée, Co...",PERSONNEL Je suis étudiante au lycée Condorcet.
3,"(Je, code, depuis, l', âge, de, 13, ans, et, j...",Je code depuis l'âge de 13 ans et j'aime créer...
4,(RÉALISATIONS),RÉALISATIONS


In [10]:
 ## Création de la colonne CV_number pour identifier les phrases de chaque CV
index2 = dataset.columns.get_loc("CV_number")
df['CV_Number'] = dataset.iloc[0,index2]       ## A MODIFIER POUR LA GENERALISATION DE L'ALGO


In [11]:
df.head()

,CV_Sentences,Sentences_CV_clean,CV_Number
0,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIASTE)",SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1
1,(PROFIL),PROFIL,CV_1
2,"(PERSONNEL, Je, suis, étudiante, au, lycée, Co...",PERSONNEL Je suis étudiante au lycée Condorcet.,CV_1
3,"(Je, code, depuis, l', âge, de, 13, ans, et, j...",Je code depuis l'âge de 13 ans et j'aime créer...,CV_1
4,(RÉALISATIONS),RÉALISATIONS,CV_1


In [12]:
## Création d'une colonne qui compte le nombre de phrase par CV
df['Sentence_line'] = df.index

In [16]:
## Création d'une colonne qui compte le nombre de tokens dans la phrase
df['Nb_tokens'] = df['CV_Sentences'].apply(lambda x : len(x))

In [17]:
df.head()

,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens
0,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIASTE)",SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1,0,6
1,(PROFIL),PROFIL,CV_1,1,1
2,"(PERSONNEL, Je, suis, étudiante, au, lycée, Co...",PERSONNEL Je suis étudiante au lycée Condorcet.,CV_1,2,8
3,"(Je, code, depuis, l', âge, de, 13, ans, et, j...",Je code depuis l'âge de 13 ans et j'aime créer...,CV_1,3,24
4,(RÉALISATIONS),RÉALISATIONS,CV_1,4,1


In [30]:
## Création d'une colonne pour connaitre la position en pourcentage de la phrase par rapport au texte entier
df['%texte_lu'] = 0
token_count = 0
for i in range(len(df)):
    token_count += df['Nb_tokens'][i]
    df['%texte_lu'][i] = round(((token_count / df['Nb_tokens'].sum()) * 100),2)   ## TROUVER COMMENT METTRE EN DECIMAL

<ipython-input-30-da0000631dca>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['%texte_lu'][i] = round(((token_count / df['Nb_tokens'].sum()) * 100),2)   ## TROUVER COMMENT METTRE EN DECIMAL


In [31]:
df.head()

,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu
0,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIASTE)",SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1,0,6,2
1,(PROFIL),PROFIL,CV_1,1,1,3
2,"(PERSONNEL, Je, suis, étudiante, au, lycée, Co...",PERSONNEL Je suis étudiante au lycée Condorcet.,CV_1,2,8,7
3,"(Je, code, depuis, l', âge, de, 13, ans, et, j...",Je code depuis l'âge de 13 ans et j'aime créer...,CV_1,3,24,18
4,(RÉALISATIONS),RÉALISATIONS,CV_1,4,1,18


In [32]:
## Création pour connaître le % de texte lu à partir de la fin de ligne
df["%texte_lu_fin_ligne"] = 0                                                       ## REVOIR LE CODE POUR LE FAIRE FONCTIONNER

nb_tokens_total = df["Nb_tokens"].sum()
i = 0
for i in range(len(df)):
    df["%texte_lu_fin_ligne"][i] += (nb_tokens_total - df['Nb_tokens'][i]) / nb_tokens_total

<ipython-input-32-4e7afcc567bf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["%texte_lu_fin_ligne"][i] += (nb_tokens_total - df['Nb_tokens'][i]) / nb_tokens_total


In [33]:
df.head()

,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne
0,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIASTE)",SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1,0,6,2,0
1,(PROFIL),PROFIL,CV_1,1,1,3,0
2,"(PERSONNEL, Je, suis, étudiante, au, lycée, Co...",PERSONNEL Je suis étudiante au lycée Condorcet.,CV_1,2,8,7,0
3,"(Je, code, depuis, l', âge, de, 13, ans, et, j...",Je code depuis l'âge de 13 ans et j'aime créer...,CV_1,3,24,18,0
4,(RÉALISATIONS),RÉALISATIONS,CV_1,4,1,18,0


In [34]:
    ## Création d'une colonne pour savoir si le mot est alphanumerique ou pas
    df["Is_alpha"] = 0
    index3 = df.columns.get_loc("Sentences_CV_clean")
    i = 0  
    for i in range(len(df)):
        doc = df.iloc[i,index3]
        doc = nlp(doc)
        list_token_is_alpha = []
        for token in doc:
            list_token_is_alpha.append(token.is_alpha)
        df["Is_alpha"][i] = list_token_is_alpha

<ipython-input-34-dd4c367e40e4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Is_alpha"][i] = list_token_is_alpha
/Users/kinn/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [35]:
df.head()

,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha
0,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIASTE)",SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1,0,6,2,0,"[True, True, True, False, True, True]"
1,(PROFIL),PROFIL,CV_1,1,1,3,0,[True]
2,"(PERSONNEL, Je, suis, étudiante, au, lycée, Co...",PERSONNEL Je suis étudiante au lycée Condorcet.,CV_1,2,8,7,0,"[True, True, True, True, True, True, True, False]"
3,"(Je, code, depuis, l', âge, de, 13, ans, et, j...",Je code depuis l'âge de 13 ans et j'aime créer...,CV_1,3,24,18,0,"[True, True, True, False, True, True, False, T..."
4,(RÉALISATIONS),RÉALISATIONS,CV_1,4,1,18,0,[True]


In [42]:
## Création de la colonne Grammaire pour connaitre le type de mots de la phrase
df["Grammar"] = 0
index = df.columns.get_loc("Sentences_CV_clean")
for i in range(len(df)):
    doc = df.iloc[i,index]
    doc = nlp(doc)
    list_token_pos = []
    for token in doc:
        list_token_pos.append(token.pos_)
        df["Grammar"][i] = list_token_pos

<ipython-input-42-256ef198810a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Grammar"][i] = list_token_pos
/Users/kinn/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [43]:
df.head()

,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha,Grammar
0,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIASTE)",SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1,0,6,2,0,"[True, True, True, False, True, True]","[NOUN, PROPN, VERB, NUM, PROPN, PROPN]"
1,(PROFIL),PROFIL,CV_1,1,1,3,0,[True],[NOUN]
2,"(PERSONNEL, Je, suis, étudiante, au, lycée, Co...",PERSONNEL Je suis étudiante au lycée Condorcet.,CV_1,2,8,7,0,"[True, True, True, True, True, True, True, False]","[NOUN, PRON, AUX, ADJ, ADP, NOUN, PROPN, PUNCT]"
3,"(Je, code, depuis, l', âge, de, 13, ans, et, j...",Je code depuis l'âge de 13 ans et j'aime créer...,CV_1,3,24,18,0,"[True, True, True, False, True, True, False, T...","[PRON, VERB, ADP, DET, NOUN, ADP, NUM, NOUN, C..."
4,(RÉALISATIONS),RÉALISATIONS,CV_1,4,1,18,0,[True],[NOUN]


In [44]:
## Création de la colonne Label qui sera notre target. Soit 1 il faut supprimer soit 0 on doit garder
df['Label'] = 0

In [45]:
df.head()

,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha,Grammar,Label
0,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIASTE)",SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1,0,6,2,0,"[True, True, True, False, True, True]","[NOUN, PROPN, VERB, NUM, PROPN, PROPN]",0
1,(PROFIL),PROFIL,CV_1,1,1,3,0,[True],[NOUN],0
2,"(PERSONNEL, Je, suis, étudiante, au, lycée, Co...",PERSONNEL Je suis étudiante au lycée Condorcet.,CV_1,2,8,7,0,"[True, True, True, True, True, True, True, False]","[NOUN, PRON, AUX, ADJ, ADP, NOUN, PROPN, PUNCT]",0
3,"(Je, code, depuis, l', âge, de, 13, ans, et, j...",Je code depuis l'âge de 13 ans et j'aime créer...,CV_1,3,24,18,0,"[True, True, True, False, True, True, False, T...","[PRON, VERB, ADP, DET, NOUN, ADP, NUM, NOUN, C...",0
4,(RÉALISATIONS),RÉALISATIONS,CV_1,4,1,18,0,[True],[NOUN],0


In [46]:
## Création de la dataset avec les CV concatenés
dataset_concat = pd.DataFrame()

In [47]:
dataset_concat = dataset_concat.append(df, ignore_index=True)

## Création d'un fichier Excel avec toutes les phrases de tous les CV
PATH = "/Users/kinn/Desktop/Projet_BlindCV/data_CV.csv"
dataset_concat.to_csv(path_or_buf = PATH, index = True)
   

In [58]:
dataset_concat.shape

(39, 10)